# README  

The goal in this script is to automate generating videos  
from images. The specific images are EOIR synthetic data.  

User to define the parameters in main() to create videos  
based on images in the output directories. Naming and  
directory structure is based on output in EOIR_Image_Generation. 

This script has been modified from the Geeks for Geeks article on   
how to [Create video using multiple images using openCV](https://www.geeksforgeeks.org/python-create-video-using-multiple-images-using-opencv/)


In [4]:
def images_to_video(filepath,data,sensor,ftype,cld,atm):

    # importing libraries
    import os
    import cv2 
    from PIL import Image 
    import csv
    import itertools

    name = []

    for (a,b,c) in itertools.product(data,ftype,sensor):
        for (x,y) in itertools.zip_longest(cld,atm):
            name.append("{}{}_{}_{}_Cld{}_Atm{}".format(filepath, a, b, c, x, y))
            
    for i in range(len(name)):
        #print(name[i])
        # Checking the current directory path
        #print(os.getcwd()) 
        
        # Folder which contains all the images
        # from which video is to be generated
        os.chdir(name[i])
        path = name[i]
        
        mean_height = 0
        mean_width = 0
        
        num_of_images = len(os.listdir('.'))
        # print(num_of_images)

        for file in os.listdir('.'):
            im = Image.open(os.path.join(path, file))
            width, height = im.size
            mean_width += width
            mean_height += height
            #im.show()   # uncomment this for displaying the image

        # Finding the mean height and width of all images.
        # This is required because the video frame needs
        # to be set with same width and height. Otherwise
        # images not equal to that width height will not get 
        # embedded into the video
        mean_width = int(mean_width / num_of_images)
        mean_height = int(mean_height / num_of_images)
        
        #print(mean_height)
        #print(mean_width)

        # Resizing of the images to give
        # them same width and height 
        for file in os.listdir('.'):
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".bmp"):
                # opening image using PIL Image
                im = Image.open(os.path.join(path, file)) 
        
                # im.size includes the height and width of image
                width, height = im.size   
                #print(width, height)
        
                # resizing 
                imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS) 
                imResize.save( file, 'JPEG', quality = 99) # setting quality
                # printing each resized image name
                #print(im.filename.split('\\')[-1], " is resized") 
        
        # removing extra parts from the video name
        discard = len(filepath) + len(data[0]) + len(ftype[0]) + 2
        
        # Video Generating function
        def generate_video():
            image_folder = '.' # make sure to use your folder
            #print(name[i][discard:])
            video_name = name[i][discard:] + ".mp4"
            os.chdir(name[i])
            
            images = [img for img in os.listdir(image_folder)
                    if img.endswith(".jpg") or
                        img.endswith(".jpeg") or
                        img.endswith(".bmp")]
            
            # Array images should only consider
            # the image files ignoring others if any.
            # Sort images by numerical value

            # Only sort for .bmp due to file numbering 
            if ftype == ['bitmap']:
                images = sorted([images for images in os.listdir('.') \
                    if images.endswith('.bmp')], \
                    key=lambda f: int(f.rsplit(os.path.extsep, 1)[0].rsplit(None,1)[-1]))
                # print(images)
            else:
                pass

            frame = cv2.imread(os.path.join(image_folder, images[0]))
        
            # setting the frame width, height width
            # the width, height of first image
            height, width, layers = frame.shape  
    
            # save one directory above the image folder 
            video = cv2.VideoWriter(str(filepath + video_name), 0, 10, (width, height)) 
        
            # Appending the images to the video one by one
            for image in images: 
                video.write(cv2.imread(os.path.join(image_folder, image))) 
            
            # Deallocating memories taken for window creation
            cv2.destroyAllWindows()
            video.release()  # releasing the video generated
        
        # Calling the generate_video function
        generate_video()

     

# User Defined

## _filepath_
File path for top level directory of all images. Each sub-  
directory is specificed with the below parameters.  

## _data_  
Data is either raw or processed. Raw data is bitmap files  
from EOIR. Processed data is normalized from the matlab  
helper functions. 

## _sensor_  
Originally from the sensor name in the scenario. Now this   
is to help identify the directory.  

## _ftype_  
Originally from the state of the data. Now this is to help  
identify the directory.  

## _cld_  
Originally from cloud settings in a run. Now this is to help  
identify the directory.  

## _atm_  
Originally from atmosphere settings in a run. Now this is to   
help identify the directory.  

In [5]:
def main():
    
    filepath = "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/output/"
    
    data    = ['processed']         #  the state of the data ['processed'] or ['raw']
    sensor  = ['LWIR','Visible']    #  names of the sensors in the scenario ['LWIR','Visible']
    ftype   = ['txt']               #  exported data format ['txt'] or ['bitmap']

    cld     = ['Off','On','On','On']  # Cloud settings per run ['Off','On','On','On']
    atm     = [0,0,1,2]               # Atmosphere settings per run [0,0,1,2]

    #generate the videos
    images_to_video(filepath,data,sensor,ftype,cld,atm)


if __name__ == '__main__':
    main()   
